In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install tqdm

In [30]:
rate = pd.read_csv(r'C:\Users\Michael\Data\title_ratings.csv.gz') 
years = [2000,2001]
errors = [ ]
rate2 = rate.copy()

In [4]:
basics = pd.read_csv(r'C:\Users\Michael\Data\title_basics.csv.gz')   
basics2 = basics.copy()

In [34]:
aka = pd.read_csv(r'C:\Users\Michael\Data\title_aka.csv.gz')   
aka2 = aka.copy()

- Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

In [6]:
with open('/Users/Michael/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [7]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [36]:
folder = "Data/"
os.makedirs(folder, exist_ok=True)
os.listdir(folder)

['final_results_fresno_pizza.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'Fresno-pizza.json',
 'title_aka.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

- As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

In [37]:
def write_json(new_data, filename): 
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [38]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
            if c['iso_3166_1'] == 'US':
                info['certification'] = c['certification']
    return info

In [39]:
def movie_year_ratings(years):

    for YEAR in tqdm_notebook(years, desc='YEARS', position=0):

        JSON_FILE = f'{folder}tmdb_api_results_{YEAR}.json'

        file_exists = os.path.isfile(JSON_FILE)

        if file_exists == False:

            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)

        df = basics.loc[ basics['startYear']==YEAR].copy()
        movie_ids = df['tconst'].copy()
        previous_df = pd.read_json(JSON_FILE)
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

        for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
            try:
                temp = get_movie_with_rating(movie_id)  
                write_json(temp,JSON_FILE)
                time.sleep(0.02)
            except Exception as e:
                errors.append([movie_id, e])
                
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    print(f"- Total errors: {len(errors)}")

In [40]:
movie_year_ratings(years)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1427 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1539 [00:00<?, ?it/s]

- Total errors: 3419


In [41]:
test_ids = ["tt0848228","tt0332280"]
results = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except: 
        pass
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28073,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.879,10077,PG-13
